In [1]:
import pandas as pd
import numpy as np
import json

# Comparison of OXA annotations across NCBI and CARD

Namely taking all the OXA from each database and running AFP and RGI on these protein sequences to see how the opposite database's tool and their own tool annotate them.

## Extract metadata from databases

In [2]:
# CARD
with open('dbs/card.json') as fh:
    card = json.load(fh)
del_keys = []
for key in card.keys():
    if key.startswith('_'):
        del_keys.append(key)

for key in del_keys:
    del card[key]
    
    
aro_to_protein_accession = {}
aro_to_drug = {}
for entry in card.values():
    if entry['model_name'].startswith('OXA'):
        accession = entry['ARO_accession']
        sequence = entry['model_sequences']['sequence']
        if len(sequence) > 1:
            print(entry)
            assert False
        else:
            sequence = sequence[list(sequence.keys())[0]]  
            aro_to_protein_accession[accession] = sequence['protein_sequence']['accession']
        
        drugs = []
        for key, category in entry['ARO_category'].items():
            if category['category_aro_class_name'] == 'Drug Class':
                drugs.append(category['category_aro_name'])
        drugs = '/'.join(sorted(drugs))
        if len(drugs) > 0:
            aro_to_drug[accession] = drugs
            
# NCBI
ncbi_db = pd.read_csv('dbs/ReferenceGeneCatalog.txt', sep='\t')
ncbi_db = ncbi_db[~ncbi_db['subclass'].isna()]
ncbi_to_drug = ncbi_db.set_index('refseq_protein_accession')['subclass'].apply(lambda x: '/'.join(sorted(x.lower().split('/'))))

## Parse NCBI cross and self-annotation

Executed via: `run.sh`

In [3]:
ncbi_oxas = []
ncbi_accessions = []
with open('ncbi_oxa.faa') as fh:
    for line in fh:
        if line.startswith('>'):
            ncbi_accessions.append(line.split('|')[1])
            ncbi_oxas.append(line.split('|')[4].replace('bla', ''))
ncbi_oxas = pd.DataFrame(index=ncbi_oxas)
ncbi_oxas['NCBI Sequence Accession'] = ncbi_accessions

ncbi_with_ncbi = pd.read_csv('rgi_afp_output/ncbi_with_ncbi_annotation.txt', sep='\t')
ncbi_with_ncbi['NCBI_ID'] = ncbi_with_ncbi['Protein identifier'].str.split('|').str.get(4).str.replace('bla', '')
ncbi_with_ncbi['NCBI_annotation'] = ncbi_with_ncbi['Gene symbol'].str.replace('bla', '')
ncbi_with_ncbi['NCBI Drug Class'] = ncbi_with_ncbi['Subclass'].apply(lambda x: '/'.join(sorted(x.lower().split('/'))))

ncbi_with_card = pd.read_csv('rgi_afp_output/ncbi_with_card_annotation.txt', sep='\t')
ncbi_with_card['NCBI_ID'] = ncbi_with_card['ORF_ID'].str.split('|').str.get(4).str.replace('bla', '')
ncbi_with_card['CARD_annotation'] = ncbi_with_card['Best_Hit_ARO']
ncbi_with_card['CARD Drug Class'] = ncbi_with_card['ARO'].apply(lambda x: aro_to_drug[str(x)] if str(x) in aro_to_drug else np.nan) 
ncbi_with_card['CARD Protein Accession'] = ncbi_with_card['ARO'].apply(lambda x: aro_to_protein_accession[str(x)] if str(x) in aro_to_protein_accession else np.nan) 


ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol'] = ncbi_with_ncbi.set_index('NCBI_ID')['NCBI_annotation']
ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 accession']  = ncbi_with_ncbi.set_index('NCBI_ID')['Accession of closest sequence']
ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 drug class']  = ncbi_with_ncbi.set_index('NCBI_ID')['NCBI Drug Class']

          
          
ncbi_oxas['RGI v5.2.1 with v3.2.4 symbol'] = ncbi_with_card.set_index('NCBI_ID')['CARD_annotation']
ncbi_oxas['RGI v5.2.1 with v3.2.4 accession'] = ncbi_with_card.set_index('NCBI_ID')['CARD Protein Accession']
ncbi_oxas['RGI v5.2.1 with v3.2.4 drug class'] = ncbi_with_card.set_index('NCBI_ID')['CARD Drug Class']


ncbi_oxas = ncbi_oxas.reset_index().rename(columns={'index': 'NCBI Sequence ID'})

ncbi_oxas.to_csv('ncbi_oxa_annotations.tsv', sep='\t')

## Parse CARD cross and self-annotation

In [4]:
card_oxas = []
with open('card_oxa.faa') as fh:
    for line in fh:
        if line.startswith('>'):
            card_oxas.append(line.split('|')[3].split()[0])
card_oxas = pd.DataFrame(index=card_oxas)

card_with_card = pd.read_csv('rgi_afp_output/card_with_card_annotation.txt', sep='\t')
card_with_card['CARD_ID'] = card_with_card['ORF_ID'].str.split('|').str.get(3).str.split().str.get(0)
card_with_card['CARD_annotation'] = card_with_card['Best_Hit_ARO']
card_with_card['CARD Protein Accession']  = card_with_card['ARO'].apply(lambda x: aro_to_protein_accession[str(x)] if str(x) in aro_to_protein_accession else np.nan) 
card_with_card['CARD Drug Class'] = card_with_card['ARO'].apply(lambda x: aro_to_drug[str(x)] if str(x) in aro_to_drug else np.nan) 


card_with_ncbi = pd.read_csv('rgi_afp_output/card_with_ncbi_annotation.txt', sep='\t')
card_with_ncbi['CARD_ID'] = card_with_ncbi['Protein identifier'].str.split('|').str.get(3).str.split().str.get(0)
card_with_ncbi['NCBI_annotation'] = card_with_ncbi['Gene symbol'].str.replace('bla', '')
card_with_ncbi['NCBI Drug Class'] = card_with_ncbi['Subclass'].apply(lambda x: '/'.join(sorted(x.lower().split('/'))))
#card_with_ncbi['NCBI accession'] = 

#ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 accession']  = ncbi_with_ncbi.set_index('NCBI_ID')['Accession of closest sequence']


card_oxas['RGI v5.2.1 with v3.2.4 symbol'] = card_with_card.set_index('CARD_ID')['CARD_annotation']
card_oxas['RGI v5.2.1 with v3.2.4 accession'] = card_with_card.set_index('CARD_ID')['CARD Protein Accession']
card_oxas['RGI v5.2.1 with v3.2.4 drug class'] = card_with_card.set_index('CARD_ID')['CARD Drug Class']

card_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol'] = card_with_ncbi.set_index('CARD_ID')['NCBI_annotation']
card_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 accession'] = card_with_ncbi.set_index('CARD_ID')['Accession of closest sequence']
card_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 drug class'] = card_with_ncbi.set_index('CARD_ID')['NCBI Drug Class']


card_oxas = card_oxas.reset_index().rename(columns={'index': 'CARD Sequence ID'})

card_oxas.to_csv('card_oxa_annotations.tsv', sep='\t')

## Comparison

In [28]:
print(f"{card_oxas[card_oxas['CARD Sequence ID'] != card_oxas['RGI v5.2.1 with v3.2.4 symbol']].shape[0]} CARD OXAs are incorrectly annotated by RGI")
print(f"{card_oxas[card_oxas['RGI v5.2.1 with v3.2.4 symbol'] != card_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol']].shape[0]} CARD OXAs are annotated as a different OXA by NCBI")
print(f"{card_oxas[card_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol'].isna()].shape[0]} CARD OXAs are missing in NCBI")
print()
print(f"{ncbi_oxas[ncbi_oxas['NCBI Sequence ID'] != ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol']].shape[0]} NCBI OXAs are incorrectly annotated by NCBI")
print(f"{ncbi_oxas[ncbi_oxas['AMRFinderPlus v3.10.40 with 2022-08-09.1 symbol'] != ncbi_oxas['RGI v5.2.1 with v3.2.4 symbol']].shape[0]} NCBI OXAs are annotated as a different OXA by CARD/RGI")
print(f"{ncbi_oxas[ncbi_oxas['RGI v5.2.1 with v3.2.4 symbol'].isna()].shape[0]} NCBI OXAs are missing in CARD")

0 CARD OXAs are incorrectly annotated by RGI
10 CARD OXAs are annotated as a different OXA by NCBI
0 CARD OXAs are missing in NCBI

1 NCBI OXAs are incorrectly annotated by NCBI
187 NCBI OXAs are annotated as a different OXA by CARD/RGI
17 NCBI OXAs are missing in CARD
